In [2]:
import numpy as np
import pandas as pd

## Explore the dataset

I use pandas for performance and interoffice sharing. I could very easily implement all of the below code using csv readers, strings + lists, etc - but I have found that sharing DFs is simply easier. Especially when some quants use R and Julia, etc.

In [34]:
sample_df = pd.read_csv('data/turnstile_130803.txt', header=None)
# sample_df.head()
sample_df[0:7] # Use [0:7] instead of .head() to show known row with NaN

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,A002,R051,02-00-00,07-27-13,00:00:00,REGULAR,4209603,1443585,07-27-13,04:00:00,...,07-28-13,00:00:00,REGULAR,4210432,1443801,07-28-13,04:00:00,REGULAR,4210472,1443805
1,A002,R051,02-00-00,07-28-13,08:00:00,REGULAR,4210490,1443821,07-28-13,12:00:00,...,07-29-13,08:00:00,REGULAR,4211176,1444042,07-29-13,12:00:00,REGULAR,4211350,1444247
2,A002,R051,02-00-00,07-29-13,16:00:00,REGULAR,4211586,1444302,07-29-13,20:00:00,...,07-30-13,14:01:00,DOOR OPEN,4213192,1444700,07-30-13,14:01:35,DOOR OPEN,4213192,1444700
3,A002,R051,02-00-00,07-30-13,14:01:46,LOGON,4213192,1444700,07-30-13,14:01:49,...,07-30-13,14:07:55,DOOR OPEN,4213192,1444700,07-30-13,14:11:56,DOOR CLOSE,4213192,1444700
4,A002,R051,02-00-00,07-30-13,16:00:00,REGULAR,4213333,1444737,07-30-13,20:00:00,...,07-31-13,16:00:00,REGULAR,4214863,1445194,07-31-13,20:00:00,REGULAR,4215656,1445251
5,A002,R051,02-00-00,08-01-13,00:00:00,REGULAR,4215894,1445274,08-01-13,04:00:00,...,08-02-13,00:00:00,REGULAR,4217489,1445773,08-02-13,04:00:00,REGULAR,4217532,1445775
6,A002,R051,02-00-00,08-02-13,08:00:00,REGULAR,4217563,1445856,08-02-13,12:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Confirm for NaN values

In [39]:
sample_df.isnull().any()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
34     True
35     True
36     True
37     True
38     True
39     True
40     True
41     True
42     True
dtype: bool

So no col names, and we can see that the 6th row has NaN data. Some rows have NaN after the 7th col. Let's explore the description fields for more info

In [40]:
print(open('data/ts_Field_Description_pre-10-18-2014.txt').read())

Field Description

C/A,UNIT,SCP,DATE1,TIME1,DESC1,ENTRIES1,EXITS1,DATE2,TIME2,DESC2,ENTRIES2,EXITS2,DATE3,TIME3,DESC3,ENTRIES3,EXITS3,DATE4,TIME4,DESC4,ENTRIES4,EXITS4,DATE5,TIME5,DESC5,ENTRIES5,EXITS5,DATE6,TIME6,DESC6,ENTRIES6,EXITS6,DATE7,TIME7,DESC7,ENTRIES7,EXITS7,DATE8,TIME8,DESC8,ENTRIES8,EXITS8


C/A = Control Area (A002)
UNIT = Remote Unit for a station (R051)
SCP = Subunit Channel Position represents an specific address for a device (02-00-00)
DATEn = Represents the date (MM-DD-YY)
TIMEn = Represents the time (hh:mm:ss) for a scheduled audit event
DEScn = Represent the "REGULAR" scheduled audit event (occurs every 4 hours)
ENTRIESn = The comulative entry register value for a device
EXISTn = The cumulative exit register value for a device



Example:
The data below shows the entry/exit register values for one turnstile at control area (A002) from 03/21/10 at 00:00 hours to 03/28/10 at 20:00 hours


A002,R051,02-00-00,03-21-10,00:00:00,REGULAR,002670738,000917107,03-21-10,04:00

## Generate known columns

Know a Max of 8 N. Also assuming that there is a typo for `EXISTS`. This is just not fun to work with so I'm taking the liberty to rename it `EXITS`

In [16]:
cols_by_id = [
    "ControlArea",
    "RemoteUnit", 
    "SCP"
]

""" add in N extra fields, just as field description says """
extra_cols = [
    "DATE",
    "TIME",
    "DESC",
    "ENTRIES",
    "EXITS", # typo on their part, corrected `EXISTS` to `EXITS` for sanity
]
max_n = 8
for i_position_in_row in xrange(max_n):
    position_in_row = str(i_position_in_row)
    for extra_col in extra_cols:
        cols_by_id.append(extra_col + "_" + position_in_row)
print(cols_by_id)

['ControlArea', 'RemoteUnit', 'SCP', 'DATE_0', 'TIME_0', 'DESC_0', 'ENTRIES_0', 'EXITS_0', 'DATE_1', 'TIME_1', 'DESC_1', 'ENTRIES_1', 'EXITS_1', 'DATE_2', 'TIME_2', 'DESC_2', 'ENTRIES_2', 'EXITS_2', 'DATE_3', 'TIME_3', 'DESC_3', 'ENTRIES_3', 'EXITS_3', 'DATE_4', 'TIME_4', 'DESC_4', 'ENTRIES_4', 'EXITS_4', 'DATE_5', 'TIME_5', 'DESC_5', 'ENTRIES_5', 'EXITS_5', 'DATE_6', 'TIME_6', 'DESC_6', 'ENTRIES_6', 'EXITS_6', 'DATE_7', 'TIME_7', 'DESC_7', 'ENTRIES_7', 'EXITS_7']


# Read in DF

This time with columns! Will still need to format data, join to figure out what ControlArea, RemoteUnit, SCP are in regards to a human readable location.

In [43]:
df_raw = pd.read_csv(
    'data/turnstile_130803.txt',
    names=cols_by_id
)

df_raw.head()

,ControlArea,RemoteUnit,SCP,DATE_0,TIME_0,DESC_0,ENTRIES_0,EXITS_0,DATE_1,TIME_1,...,DATE_6,TIME_6,DESC_6,ENTRIES_6,EXITS_6,DATE_7,TIME_7,DESC_7,ENTRIES_7,EXITS_7
0,A002,R051,02-00-00,07-27-13,00:00:00,REGULAR,4209603,1443585,07-27-13,04:00:00,...,07-28-13,00:00:00,REGULAR,4210432,1443801,07-28-13,04:00:00,REGULAR,4210472,1443805
1,A002,R051,02-00-00,07-28-13,08:00:00,REGULAR,4210490,1443821,07-28-13,12:00:00,...,07-29-13,08:00:00,REGULAR,4211176,1444042,07-29-13,12:00:00,REGULAR,4211350,1444247
2,A002,R051,02-00-00,07-29-13,16:00:00,REGULAR,4211586,1444302,07-29-13,20:00:00,...,07-30-13,14:01:00,DOOR OPEN,4213192,1444700,07-30-13,14:01:35,DOOR OPEN,4213192,1444700
3,A002,R051,02-00-00,07-30-13,14:01:46,LOGON,4213192,1444700,07-30-13,14:01:49,...,07-30-13,14:07:55,DOOR OPEN,4213192,1444700,07-30-13,14:11:56,DOOR CLOSE,4213192,1444700
4,A002,R051,02-00-00,07-30-13,16:00:00,REGULAR,4213333,1444737,07-30-13,20:00:00,...,07-31-13,16:00:00,REGULAR,4214863,1445194,07-31-13,20:00:00,REGULAR,4215656,1445251


## Figure out ControlArea + RemoteUnit

In [44]:
stations = pd.read_excel('data/Remote-Booth-Station.xls')
stations.head()

,Remote,Booth,Station,Line Name,Division
0,R001,A060,WHITEHALL ST,R1,BMT
1,R001,A058,WHITEHALL ST,R1,BMT
2,R001,R101S,SOUTH FERRY,R1,IRT
3,R002,A077,FULTON ST,ACJZ2345,BMT
4,R002,A081,FULTON ST,ACJZ2345,BMT


,Remote,Booth,Station,Line Name,Division
0,R001,A060,WHITEHALL ST,R1,BMT
1,R001,A058,WHITEHALL ST,R1,BMT
2,R001,R101S,SOUTH FERRY,R1,IRT
3,R002,A077,FULTON ST,ACJZ2345,BMT
4,R002,A081,FULTON ST,ACJZ2345,BMT


In [270]:
cols_by_id = [
    "ControlArea",
    "RemoteUnit", 
    "SCP"
]

from pprint import pprint

""" add in N extra fields, just as field description says """
extra_cols = [
    "DATE",
    "TIME",
    "DESC",
    "ENTRIES",
    "EXITS", # typo on their part
]
max_n = 8 #max([(len(row) - len(cols_by_id)) / len(extra_cols) for row in raw_data])
for i_position_in_row in xrange(max_n):
    i_position_in_row = str(i_position_in_row)
    for extra_col in extra_cols:
        cols_by_id.append(extra_col + "_" + i_position_in_row)
print(cols_by_id)

""" Make dict for easy lookups"""
cols = {}
for i_name, name in enumerate(cols_by_id):
    cols[name] = i_name

""" get row into a structured format """
cleaner_rows = []
for i_row, messy_row in enumerate(raw_data):
#     if(i_row > 10): break
    
    cleaned_row = {}
    for i_val, val in enumerate(messy_row):        
#         print(i_val, val, cols_by_id[i_val])
        cleaned_row[cols_by_id[i_val]] = val.rstrip() # end of rows may have trailing spaces and endlines
    cleaned_row['row_id'] = i_row + 1
#     pprint(cleaned_row)
    cleaner_rows.append(cleaned_row)
    
df_combined_rows = pd.DataFrame(cleaner_rows)

['ControlArea', 'RemoteUnit', 'SCP', 'DATE_0', 'TIME_0', 'DESC_0', 'ENTRIES_0', 'EXITS_0', 'DATE_1', 'TIME_1', 'DESC_1', 'ENTRIES_1', 'EXITS_1', 'DATE_2', 'TIME_2', 'DESC_2', 'ENTRIES_2', 'EXITS_2', 'DATE_3', 'TIME_3', 'DESC_3', 'ENTRIES_3', 'EXITS_3', 'DATE_4', 'TIME_4', 'DESC_4', 'ENTRIES_4', 'EXITS_4', 'DATE_5', 'TIME_5', 'DESC_5', 'ENTRIES_5', 'EXITS_5', 'DATE_6', 'TIME_6', 'DESC_6', 'ENTRIES_6', 'EXITS_6', 'DATE_7', 'TIME_7', 'DESC_7', 'ENTRIES_7', 'EXITS_7']


In [271]:
""" convert rows with multiple sets of data to be single rows per set. More uniform analysis """

sanitized_rows = []
for i_row, row in df_combined_rows.iterrows():
    
    for i_time_interval in xrange(max_n):
        time_interval = str(i_time_interval)

        """ only those which do not have NaN as this grouping """
        if pd.notnull(row['DATE_' + time_interval]) is False:
#             print(row['row_id'], i_time_interval, 'ENTRIES_' + time_interval)
            continue

        sanitized_row = {
            "RemoteUnit": row["RemoteUnit"],
            "SCP": row["SCP"],
            "ControlArea": row["ControlArea"],
            "DATE": row['DATE_' + time_interval],
            "ENTRIES": row["ENTRIES_" + time_interval],
            "EXITS": row["EXITS_" + time_interval],
            "DESC": row['DESC_' + time_interval],
            "TIME": row['TIME_' + time_interval],
            
            "row_id": row['row_id'],
            "interval": i_time_interval,
        }
        
        
        sanitized_rows.append(sanitized_row)
        
sanitized_df = pd.DataFrame(sanitized_rows)



## Format Data

Let's correct the dtypes before we split into a more logical DF, removing `N` cols and making rows per `N`. Not required for analysis, but I think this will make the interactive analysis easier and uniform. 

In [272]:
""" Format cols """

df = sanitized_df

""" datetimes """  
df['DATETIME'] = pd.to_datetime(
    df['DATE'] + df['TIME'],
    format="%m-%d-%y%X"
)
df['DATE'] = pd.to_datetime(
    df['DATE'],
    format="%m-%d-%y"
)

del df['TIME']

""" numeric """
df['ENTRIES'] = df['ENTRIES'].convert_objects(
    convert_numeric=True
)
df['EXITS'] = df['EXITS'].convert_objects(
    convert_numeric=True
)


In [277]:
stations.head()

,Remote,Booth,Station,Line Name,Division
0,R001,A060,WHITEHALL ST,R1,BMT
1,R001,A058,WHITEHALL ST,R1,BMT
2,R001,R101S,SOUTH FERRY,R1,IRT
3,R002,A077,FULTON ST,ACJZ2345,BMT
4,R002,A081,FULTON ST,ACJZ2345,BMT


In [287]:
stations.loc[stations['Booth'] == 'A060']

,Remote,Booth,Station,Line Name,Division
0,R001,A060,WHITEHALL ST,R1,BMT


In [308]:
merged = pd.merge(
    df,
    stations,
    left_on=["ControlArea", "RemoteUnit"],
    right_on=["Booth", "Remote"], # Remote Booth
    how="left"
)

In [288]:
stations.loc[stations['Remote'] == 'R051']

,Remote,Booth,Station,Line Name,Division
115,R051,R245,59 ST,456NQR,IRT
116,R051,R245A,59 ST,456NQR,IRT
117,R051,A002,LEXINGTON AVE,456NQR,BMT


In [303]:
len(df)

217844

In [283]:
df.loc[df['RemoteUnit'] == 'R001']

,ControlArea,DATE,DESC,ENTRIES,EXITS,RemoteUnit,SCP,interval,row_id,DATETIME
10547,A058,2013-07-27,REGULAR,192708,335875,R001,01-00-00,0,1440,2013-07-27 01:00:00
10548,A058,2013-07-27,REGULAR,192708,335881,R001,01-00-00,1,1440,2013-07-27 05:00:00
10549,A058,2013-07-27,REGULAR,192710,335882,R001,01-00-00,2,1440,2013-07-27 09:00:00
10550,A058,2013-07-27,REGULAR,192710,335882,R001,01-00-00,3,1440,2013-07-27 13:00:00
10551,A058,2013-07-27,REGULAR,192710,335884,R001,01-00-00,4,1440,2013-07-27 17:00:00
10552,A058,2013-07-27,REGULAR,192710,335885,R001,01-00-00,5,1440,2013-07-27 21:00:00
10553,A058,2013-07-28,REGULAR,192710,335889,R001,01-00-00,6,1440,2013-07-28 01:00:00
10554,A058,2013-07-28,REGULAR,192712,335890,R001,01-00-00,7,1440,2013-07-28 05:00:00
10555,A058,2013-07-28,REGULAR,192712,335895,R001,01-00-00,0,1441,2013-07-28 09:00:00
10556,A058,2013-07-28,REGULAR,192712,335899,R001,01-00-00,1,1441,2013-07-28 13:00:00


In [273]:
df.loc[sanitized_df['row_id'] == 7]

,ControlArea,DATE,DESC,ENTRIES,EXITS,RemoteUnit,SCP,interval,row_id,DATETIME
48,A002,2013-08-02,REGULAR,4217563,1445856,R051,02-00-00,0,7,2013-08-02 08:00:00
49,A002,2013-08-02,REGULAR,4217715,1446068,R051,02-00-00,1,7,2013-08-02 12:00:00
50,A002,2013-08-02,REGULAR,4218123,1446120,R051,02-00-00,2,7,2013-08-02 16:00:00
51,A002,2013-08-02,REGULAR,4218917,1446183,R051,02-00-00,3,7,2013-08-02 20:00:00


In [274]:
df.dtypes

ControlArea            object
DATE           datetime64[ns]
DESC                   object
ENTRIES                 int64
EXITS                   int64
RemoteUnit             object
SCP                    object
interval                int64
row_id                  int64
DATETIME       datetime64[ns]
dtype: object

In [275]:
df.describe()

,ENTRIES,EXITS,interval,row_id
count,2.178440e+05,2.178440e+05,217844.000000,217844.000000
mean,5.600449e+06,3.309910e+06,3.391932,14723.317989
std,3.551615e+07,3.494169e+07,2.296694,8490.042628
min,-9.314769e+08,-8.786480e+08,0.000000,1.000000
25%,3.454710e+05,2.033342e+05,1.000000,7364.000000
50%,2.027753e+06,1.233320e+06,3.000000,14723.000000
75%,5.163563e+06,3.670348e+06,5.000000,22061.000000
max,9.168487e+08,8.624322e+08,7.000000,29427.000000


In [269]:
""" stats """

from datetime import datetime

date_min = datetime(2013, 8, 1)
date_max = datetime(2013, 8, 2)

august_1st = df.loc[
    (df['DATETIME'] >= date_min) & (df['DATETIME'] < date_max)
]

august_1st['ENTRIES'].sum() + august_1st['EXITS'].sum()

285666532693